In [1]:
import librosa
import librosa.display
import soundfile as sf

import numpy as np
import matplotlib.pyplot as plt


In [8]:
import wave

data = wave.open('../audio_processing/samples/sample_b.wav', mode = 'rb')
params = data.getparams()
print(params)

_wave_params(nchannels=2, sampwidth=2, framerate=44100, nframes=141120, comptype='NONE', compname='not compressed')


In [9]:
# Load the stereo audio file
audio_a, sr_a = librosa.load('../audio_processing/samples/sample_a.wav', mono = False)

audio_b, sr_b = librosa.load('../audio_processing/samples/sample_b.wav', mono = False)

# Split the stereo audio into two mono channels
# channel_left = audio[:,0]
# channel_right = audio[:,1]
sr_a

22050

In [4]:
# # test = librosa.feature.melspectrogram(y=audio, sr=sr)

# size = 2048
# m = 1/32

# # Passing through arguments to the Mel filters
# D = librosa.stft(audio, n_fft=size, hop_length = int(size*m), win_length = int(size*m))  # STFT of y
# S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)


# fig, ax = plt.subplots()
# img = librosa.display.specshow(S_db[0], n_fft=size, hop_length = int(size), win_length = int(size*m))
# fig.patch.set_visible(False)

In [15]:
interpolated_signals = [audio_a + (audio_b - audio_a) * i / 14 for i in range(1, 15)]

for i, y in enumerate(interpolated_signals):
    sf.write('../audio_processing/output/interpolated_signal_{}.wav'.format(i), y, sr_a, format='WAV', subtype='PCM_24')
    # sf.write('../audio_processing/output/interpolated_signal_{}.wav'.format(i), y, sr_a, subtype='PCM_16')

ValueError: Unknown format: 'WAVE'